# Домашнее задание к лекции «Понятие класса»

## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [1]:
import requests
import time

In [2]:
def valute_name():
    """
    Функция возвращает название валюты (поле ‘Name’) с максимальным значением курса 
    с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js.
    """
    result = []
    for i in range(5): # Цикл используется для избежания ошибки "падения" сервиса при загрузке внешних данных
        try:
            roster = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
        except requests.Timeout as e:
            print(f'Ожидайте загрузки данных. Жду {2**i}c')
            time.sleep(2**i)
    for valute in roster['Valute']: #Формируем список, состоящий из названия валюты и его курса
        result.append([roster['Valute'][valute]['Name'], roster['Valute'][valute]['Value']]) 
    return max(result, key=lambda x: x[1])[0]

In [3]:
valute_name()

'СДР (специальные права заимствования)'

## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [4]:
class Rate:
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate(format_='value', diff=False).make_format('EUR')
        79.4966
        
        Rate(format_='value', diff=True).make_format('EUR')
        -0.3561
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value' and self.diff == False:
                return response[currency]['Value']
            if self.format == 'value' and self.diff == True:
                return round((float(response[currency]['Value']) - float(response[currency]['Previous'])), 4)
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [5]:
r = Rate(format_='full')

In [6]:
r.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 73.3057,
 'Previous': 73.6618}

In [7]:
r = Rate(diff=True)

In [8]:
r.usd()

-0.3561

In [9]:
r = Rate(format_='full', diff=True)

In [10]:
r.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 73.3057,
 'Previous': 73.6618}

## Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [11]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [12]:
class Designer(Employee):
    def __init__(self, name, seniority=4):
        super().__init__(name, seniority)
    
    def upgrade_for_designer(self):
        """
        Повышает уровень дизайнеров при получении международной премии.
        Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла
        """
        self.seniority += 2 #Повышает количество баллов при получении международной премии
        if self.seniority % 7 == 0 or self.seniority % 7 == 1:
            self.grade_up() #Повышает уровень сотрудника за каждые 7 баллов
        return self.publish_grade()

In [13]:
Maksim = Designer('Максим', 0)

In [14]:
for i in range(22):
    Maksim.upgrade_for_designer()

Максим 1
Максим 1
Максим 1
Максим 2
Максим 2
Максим 2
Максим 3
Максим 3
Максим 3
Максим 3
Максим 4
Максим 4
Максим 4
Максим 5
Максим 5
Максим 5
Максим 5
Максим 6
Максим 6
Максим 6
Максим 7
Максим 7


In [15]:
Elena = Designer('Елена')

In [16]:
for i in range(22):
    Elena.upgrade_for_designer()

Елена 1
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 5
Елена 5
Елена 5
Елена 5
Елена 6
Елена 6
Елена 6
Елена 7
Елена 7
Елена 7
Елена 7
